In [1]:
from langchain.chains import LLMChain
from gpu_llm import GpuLLM, LLMConfig

conf = LLMConfig(max_length=1000,
                do_sample=False,
                temperature=0.0,
                top_k=0,
                top_p=1,
                repetition_penalty=1.0,
                early_stopping=False,
                # _model = None,
                # _stop_regex = None,
                # stopping_criteria = "</s>",
                )


open_ai = GpuLLM(config=conf,
                    seed=153513,
                model_name = "/lustre/scratch/public/falcon-40b-instruct",
                device="auto",
                skip_validation=True,
                # stop_msgs=["</s>"],
                )

open_ai._initialize()

# from langchain.llms import HuggingFacePipeline

# open_ai = HuggingFacePipeline.from_model_id(
# model_id= "/lustre/scratch/public/falcon-40b-instruct",
# device="auto",
# task="text-generation", model_kwargs={"temperature": 0},
# # streaming=True
# )



Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [2]:
from langchain.agents import load_tools, initialize_agent,AgentType,Tool
from langchain.utilities import GoogleSearchAPIWrapper


# search = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY,google_cse_id=GOOGLE_CSE_ID)
# TODO: add coding tools
tools = [
    # Tool(
    # name="Google",
    # func=search.run,
    # description="Use to perfrom Google search through out the internet"
    # ),
    Tool(name="get_Total_Income",
        func=lambda x : "user had earn a total of 1700$",
        description="Use to get User Total Income"
        ),
    Tool(name="get_Income_Source",
        func=lambda x : "User have a following source of income recently. food, clothes, cars",
        description="Use to get User Income Source"
        ),
    Tool(name="get_Total_Expense",
        func=lambda x : "user had spend a total of 250$",
        description="Use to get User Total Expense"
        ),
    Tool(name="get_Expense",
        func=lambda x : "user had spend 10$ by buying food, 40$ for rent, 100$ for clothes, 100$ for cars",
        description="Use to get User expense"
        ),
    Tool(name="get_Income",
        func=lambda x : "user had earn 100$ by selling food, 600$ by selling clothes, 1000$ by selling cars",
        description="Use to get User income"
        ),
    
        
] + load_tools(
    ['llm-math'], llm=open_ai,
    )    


In [3]:
from langchain.agents import ZeroShotAgent, AgentExecutor
from langchain.memory import ConversationBufferWindowMemory

agent_prefix = """You are a financial assistance. your job is to answer the user question and Think logicly! You should use tools to help you answer the question.
You have access to the following tools:""" 
agent_suffix = """Begin!
Question: {input}
{agent_scratchpad}"""


# {question_history}

tool_names = [tool.name for tool in tools]
agent_convhis = ConversationBufferWindowMemory(k=2,memory_key="question_history",output_key='output')
agent_prompt = ZeroShotAgent.create_prompt(tools=tools,
                                           prefix=agent_prefix, 
                                           suffix=agent_suffix,
                                           input_variables=["input",'agent_scratchpad','question_history'])
agent_custom = ZeroShotAgent(llm_chain=LLMChain(llm=open_ai,prompt=agent_prompt),allowed_tools=tool_names,return_intermediate_steps=True
                             ,max_iterations=1)
agent_commit = AgentExecutor.from_agent_and_tools(agent=agent_custom,tools=tools,
                                                verbose=True,
                                                  # memory=agent_convhis,
                                                  # handle_parsing_errors=True,
                                                  return_intermediate_steps=True)

In [5]:
agent_commit({'input':"what is my total expense?"}) 


/home/bsabsri/miniconda3/envs/rnd-project/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/bsabsri/miniconda3/envs/rnd-project/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new AgentExecutor chain...
You are a financial assistance. your job is to answer the user question and Think logicly! You should use tools to help you answer the question.
You have access to the following tools:

Google: Use to perfrom Google search through out the internet
get_Total_Income: Use to get User Total Income
get_Income_Source: Use to get User Income Source
get_Total_Expense: Use to get User Total Expense
get_Expense: Use to get User expense
get_Income: Use to get User income
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Google, get_Total_Income, get_Income_Source, get_Total_Expense, get_Expense, get_Income, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I

/home/bsabsri/miniconda3/envs/rnd-project/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/bsabsri/miniconda3/envs/rnd-project/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Thought: I need to get the total expense
Action: get_Total_Expense
Action Input:
Observation:
Observation: user had spend a total of 250$
Thought:You are a financial assistance. your job is to answer the user question and Think logicly! You should use tools to help you answer the question.
You have access to the following tools:

Google: Use to perfrom Google search through out the internet
get_Total_Income: Use to get User Total Income
get_Income_Source: Use to get User Income Source
get_Total_Expense: Use to get User Total Expense
get_Expense: Use to get User expense
get_Income: Use to get User income
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Google, get_Total_Income, get_Income_Source, get_Total_Expense, get_Expense, get_Income, Calculator]
Action Input: the input to the action
Observation:

{'input': 'what is my total expense?',
 'output': '250$',
 'intermediate_steps': [(AgentAction(tool='get_Total_Expense', tool_input='Observation:', log='Thought: I need to get the total expense\nAction: get_Total_Expense\nAction Input:\nObservation:'),
   'user had spend a total of 250$')]}

In [ ]:
# from langchain import PromptTemplate
# prompt = PromptTemplate("""You are a financial assistance. your job is to answer the user question.
# """)

In [ ]:
# prediction = LLMChain(llm=open_ai,prompt=prompt)